<b><h3>Importing libraries</h3></b>

In [48]:
!conda update scikit-learn --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [49]:
import os, random
import numpy as np
import pandas as pd
import PIL
import keras
import itertools
from PIL import Image


from sklearn.utils import shuffle 
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

<b><h3>Extracting zip files to classify images</h3></b>

In [50]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile((os.environ['DSX_PROJECT_DIR']+'/datasets/distractions.zip'),'r')
paths = zip_ref.namelist() 
print(paths)
classes_required=[]
for path in paths:
    zip_ref.extract(path)
    temp=path.split('/') 
    if len(temp) == 3:
        if temp[1] not in classes_required:
            classes_required.append(temp[1])
print(classes_required)
zip_ref.close()

['distractions/c1/', 'distractions/c1/img_115.jpg', 'distractions/c1/img_149.jpg', 'distractions/c1/img_357.jpg', 'distractions/c1/img_448.jpg', 'distractions/c1/img_6.jpg', 'distractions/c2/', 'distractions/c2/img_186.jpg', 'distractions/c2/img_253.jpg', 'distractions/c2/img_258.jpg', 'distractions/c2/img_271.jpg', 'distractions/c2/img_94.jpg', 'distractions/c3/', 'distractions/c3/img_204.jpg', 'distractions/c3/img_207.jpg', 'distractions/c3/img_216.jpg', 'distractions/c3/img_244.jpg', 'distractions/c3/img_5.jpg']
['c1', 'c2', 'c3']


<b><h3>Defining the testing, training and valdiation sets</h3></b>

In [51]:
batch_size_train = 3
batch_size_val = 15
batch_size_test = 15
num_classes= 3
intereseted_folder='distractions'
STANDARD_SIZE=(224,224)

In [52]:
'''Converting Data Format according to the backend used by Keras
'''
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

<b><h3>Extracting training dataset...</h3></b> Do this only once

In [53]:
import zipfile
zip_ref = zipfile.ZipFile(os.environ['DSX_PROJECT_DIR']+'/datasets/distractions.zip', 'r')
zip_ref.extract((os.environ['DSX_PROJECT_DIR']+'/datasets/distractions/c1'))
zip_ref.close()

KeyError: "There is no item named '/user-home/1002/DSX_Projects/Distraction Detection/datasets/distractions/c1' in the archive"

In [ ]:

'''Input the Training Data
''' 
import zipfile
#zip_ref = zipfile.ZipFile(os.environ['DSX_PROJECT_DIR']+'/datasets/distractions.zip', 'r')
#zip_ref.extract((os.environ['DSX_PROJECT_DIR']+'/datasets/'))
#zip_ref.close()
train_path = os.environ['DSX_PROJECT_DIR']+'/datasets/distractions' 
print(train_path)
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_train)
type(train_batches)

In [ ]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile((os.environ['DSX_PROJECT_DIR']+'/datasets/val_data.zip'),'r')
paths = zip_ref.namelist() 
print(paths)
classes_required=[]
for path in paths:
    zip_ref.extract(path)
    temp=path.split('/') 
    if len(temp) == 3:
        if temp[1] not in classes_required:
            classes_required.append(temp[1])
print(classes_required)
zip_ref.close()

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile(os.environ['DSX_PROJECT_DIR']+'/datasets/val_data.zip', 'r')
zip_ref.extractall((os.environ['DSX_PROJECT_DIR']+'/datasets/'))
zip_ref.close()

In [ ]:

'''Input the Validation Data
'''

val_path = os.environ['DSX_PROJECT_DIR']+'/datasets/val_data'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_val)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile(os.environ['DSX_PROJECT_DIR']+'/datasets/test_data.zip', 'r')
zip_ref.extractall((os.environ['DSX_PROJECT_DIR']+'/datasets/'))
zip_ref.close()

In [ ]:
'''Input the Test Data
'''
test_path = os.environ['DSX_PROJECT_DIR']+'/datasets/test_data'
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_test)

In [ ]:
test_imgs, test_labels = next(test_batches)
test_labels

In [ ]:
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

<b><h3>Create the CNN model</h3></b>

In [ ]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()

In [ ]:
type(vgg16_model) #This is a Keras Functional API need to convert to sequential
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)

In [ ]:
model.layers.pop()
model.summary()

In [ ]:
for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [ ]:
model.add(Dense(3, activation='sigmoid')) # Add the last layer
model.summary()

In [ ]:
# Complie the model
model.compile(Adam(lr=.00015), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=20, 
                    validation_data=val_batches, validation_steps=20, epochs=5, verbose=1)

<b><h3>Extracting dataset for testing out model</h3></b>

In [ ]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile((os.environ['DSX_PROJECT_DIR']+'/datasets/sample3.zip'),'r')
paths = zip_ref.namelist()

print(paths)
for path in paths:
    print(zip_ref.extract(path))
zip_ref.close()

<b><h3>Testing the model and getting the predictions</h3></b>

In [ ]:
X_test=[] 
def convert_to_image(X):
    '''Function to convert all Input Images to the STANDARD_SIZE and create Training Dataset
    '''
    for f in paths:
        #fobj=get_file(f)
        #print(type(fobj))predictions= model.predict(X_test)
        if os.path.isdir(f):
            continue
        img= PIL.Image.open(f)
        img = img.resize(STANDARD_SIZE) 
        img_test = img
        img=np.array(img)
        X.append(img)
        #print(X_train)
    #print(len(X_train))
    return X
X_test=np.array(convert_to_image(X_test))
datagen.fit(X_test)

In [ ]:
predictions= model.predict(X_test)
predictions

In [ ]:
# y_pred=[]
# for i in range(len(predictions)):
#     y_pred.append(np.argmax(predictions[i]))
# print(y_pred) 
# print(i) 
# print(y_pred[0])
# print(paths)
# print(len(predictions)) 
# print(np.argmax(predictions[i])) 
# j = 0
# for i in y_pred:
#     print(paths[y_pred[j]])
#     j = j + 1 



y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))
y_pred
j = 0
for i in y_pred:
    print(paths[y_pred[j]])
    j = j + 1

In [ ]:
#print(classes_required)
index= classes_required.index('c1') #Edit this class index here to retrieve image of that class 
index1= classes_required.index('c2') #Edit this class index here to retrieve image of that class
index2= classes_required.index('c3') #Edit this class index here to retrieve image of that class

for i in range(len(y_pred)):
    if y_pred[i] == index:
        print("Image that has been classified as a c1 is: ", paths[i]) 
    elif y_pred[i] == index1:
        print("Image that has been classified as a c2 is: ", paths[i]) 
    elif y_pred[i] == index2:
        print("Image that has been classified as a c3 is: ", paths[i])

In [ ]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)
predictions

In [ ]:
predictions
y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))
print(y_pred)
#plots(test_imgs, titles=y_pred)

ctr=0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        ctr=ctr+1
res = ctr/len(y_pred)*100
print(res)

<b><h3>Deploying the model as a WML model</h3></b>

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
                    "url": "https://52.116.34.53","username": "contestant2","password": "lkjhgf65", "instance_id": "icp"               
 }
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
model_props = {client.repository.ModelMetaNames.NAME: "A sample model", client.repository.ModelMetaNames.FRAMEWORK_NAME: 'tensorflow',
        client.repository.ModelMetaNames.FRAMEWORK_VERSION: '1.12',
        client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
        client.repository.ModelMetaNames.RUNTIME_VERSION: '3.5',
        client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [
         {"name": "keras", "version": "2.2.4"}
    ]}   
model_details = client.repository.store_model(model,  model_props)